In [57]:
import pandas as pd
import numpy as np
import os
import fnmatch
import requests
from lxml import etree

In [ ]:
# Income statement
def incomeSta(code,filename,method):
    r1 = requests.get('http://www.aastocks.com/sc/ipo/CompanyFundamental.aspx?CFType=6&view=1&symbol=%s'%code).text
    html1 = etree.HTML(r1)
    income = html1.xpath('//table[@id="PL"]//tr')
    for i in income:
        info = ''
        if len(i.findall('.//td')) >2:
            for j in i.findall('.//*'):
                if str(j.text) != 'None':
                    info += (str(j.text)+'|')
            info += '\n'
            write_to_file(filename,method,info.replace('\xa0',''))
# cashflow
def cf(code,filename,method):
    r2 = requests.get('http://www.aastocks.com/sc/ipo/CompanyFundamental.aspx?CFType=7&view=1&symbol=%s'%code).text
    html2 = etree.HTML(r2)
    cashflow = html2.xpath('//table[@id="CF"]//tr')
    for i in cashflow:
        info = ''
        if len(i.findall('.//td')) >2:
            for j in i.findall('.//*'):
                if str(j.text) != 'None':
                    info += (str(j.text)+'|')
            info += '\n'
            write_to_file(filename,method,info.replace('\xa0',''))
# balance sheet
def bs(code,filename,method):
    r3 = requests.get('http://www.aastocks.com/sc/ipo/CompanyFundamental.aspx?CFType=8&view=1&symbol=%s'%code).text
    html3 = etree.HTML(r3)
    bs = html3.xpath('//table[@id="BS"]//tr')
    for i in bs:
        info = ''
        if len(i.findall('.//td')) >2:
            for j in i.findall('.//*'):
                if str(j.text) != 'None':
                    info += (str(j.text)+'|')
            info += '\n'
            write_to_file(filename,method,info.replace('\xa0',''))

In [ ]:
# 爬取财务信息三张表
# 每只股票的三张表分别存放于不同文件：资产负债表: bs_股票代码.txt 利润表：is_股票代码.txt 现金流量表:cf_股票代码.txt
# hoffman
if __name__=='__main__':
    with open('stocks.txt') as f:
    d = f.readlines()
    codes = [i.split(',')[1] for i in d[0].split('|')[:-1]]
    for code in codes[1:2]:
        incomeSta(code,'is_%s.txt'%code,'a')
        cf(code,'cf_%s.txt'%code,'a')
        bs(code,'bs_%s.txt'%code,'a')

In [109]:
# 将所有股票的资产负债表信息汇总到bs.txt,利润表信息汇总到is.txt
# 文件stocks.txt存放了2014.9.19-2019.7.5日所有ipo的股票代码和ipo日期，每只股票用'|'隔开，日期和股票代码用‘,’隔开，如：2019/06/18,1935|2019/06/17,3877|
stocks = open('stocks.txt','r').read().split('|')

In [103]:
def income_s(code,ipoday):
    # construct filename
    fname = 'is_'+code+'.txt'
    # read is file
    income = open(fname,'r',encoding='UTF-8').read().split('\n')
    # write code and ipoday
    with open('is.txt','a',encoding='UTF-8') as f:
        f.write(code+'|'+ipoday+'|')
        
    header = income[0].split('|')
    n = len(header)-2 # 列数
    
    # i： 从左往右第一列ipo日以前的数据
    for i in range(1,n+1):
        if header[i]<=ipoday:
            break

    currency = income[1].split('|')[1]
    sale = income[3].split('|')
    opt_profit = income[4].split('|')
    holder_profit = income[10].split('|')

    for j in range(i,n+1):
        with open('is.txt','a',encoding='UTF-8') as f:
            f.write(sale[j]+'|'+opt_profit[j]+'|'+holder_profit[j]+'|')

    with open('is.txt','a',encoding='UTF-8') as f:
        f.write(currency + '\n')


In [104]:
def bs(code,ipoday):
    # construct filename
    fname = 'bs_'+code+'.txt'
    # read is file
    balance = open(fname,'r',encoding='UTF-8').read().split('\n')
    # write code and ipoday
    with open('bs.txt','a',encoding='UTF-8') as f:
        f.write(code+'|'+ipoday+'|')
        
    header = balance[0].split('|')
    n = len(header)-2 # 列数
    
    # i： 从左往右第一列ipo日以前的数据
    for i in range(1,n+1):
        if header[i]<=ipoday:
            break
    asset = balance[7].split('|')
    cash = balance[16].split('|')
    liability = balance[18].split('|')
    
    for j in range(i,n+1):
        with open('bs.txt','a',encoding='UTF-8') as f:
            f.write(asset[j]+'|'+cash[j]+'|'+liability[j]+'|')

    with open('bs.txt','a',encoding='UTF-8') as f:
        f.write('\n')

In [ ]:
# 开始收集财务数据：分别生成bs.txt和is.txt
if __name__=='__main__':
    for stock in stocks[1:]:
        code = stock.split(',')[1]
        ipoday = stock.split(',')[0][:7]
        income_s(code,ipoday)
        bs(code,ipoday)

In [66]:
# 检查抓取的词条是否对应所要抓取的内容，另用check.py在hoffman中跑
# 结果：所有银行股的抓取结果有误，其他的无误。可以考虑在最后的结果中剔除银行股
'''import pandas as pd
import numpy as np
import os
import fnmatch
# check function
def check_bs(bs):
    return(len(bs)==21,bs[7].split('|')[0]=='总资产',bs[16].split('|')[0]=='现金及银行结存',bs[18].split('|')[0]=='总债项')

def check_is(income):
    return(len(income)==21,income[3].split('|')[0]=='营业额',income[4].split('|')[0]=='经营溢利',income[10].split('|')[0]=='股东应占溢利')

# start to check
# read file names
filels = os.listdir()
pattern_bs = 'bs_*.txt'
pattern_is = 'is_*.txt'
# add headers
with open('check_bs.txt','a') as f:
            f.write(str(('lenth','总资产','现金及银行结存','总债项'))+'\n')
with open('check_is.txt','a') as f:
            f.write(str(('lenth-is','营业额','经营溢利','股东应占溢利'))+'\n')
# check
for file in filels:
    if fnmatch.fnmatch(file,pattern_bs):
        bs = open(file,'r',encoding='UTF-8').read().split('\n')
        with open('check_bs.txt','a') as f:
            f.write(str(check_bs(bs))+'\n')
    elif fnmatch.fnmatch(file,pattern_is):
        income = open(file,'r',encoding='UTF-8').read().split('\n')
        with open('check_is.txt','a') as f:
            f.write(str(check_is(income))+'\n')'''